In [273]:
import numpy as np
from numpy import linalg as LA
from numpy.linalg import inv
from scipy.linalg import rq
import math

In [274]:
# x=np.array([[0,1],[2,3]])
# X=np.array([[0,1,1],[1,2,3]])

In [275]:
X = np.array([
    [4.74, 4.74, 0],
    [7.11, 7.11, 0],
    [7.11, 11.85, 0],
    [11.85, 11.85, 4.74],
    [11.85, 7.11, 4.74],
    [11.85, 4.74, 7.11]
]
)

In [276]:
x = np.array([
    [356, 836],
    [450, 815],
    [581, 886],
    [682, 700],
    [556, 804],
    [528, 532]
]
)

In [277]:
def normalizing_params(pts):
    centroid=np.mean(pts,axis=0)
    avg_distance=np.mean(LA.norm(pts-centroid, axis=1))
    return centroid,avg_distance


In [278]:
def normalizing_matrix(pts):
    c,d=normalizing_params(pts)
    n=pts.shape[1]
    if n==2:
        d=d/math.sqrt(2)
        mat=np.array([[1/d,0,-c[0]/d],[0,1/d,-c[1]/d],[0,0,1]])
    else:
        d=d/math.sqrt(3)
        mat=np.array([[1/d,0,0,-c[0]/d],[0,1/d,0,-c[1]/d],[0,0,1/d,-c[2]/d],[0,0,0,1]])
    return mat

In [279]:
def homo_coord(pts):
    rows=pts.shape[0]
    pts=np.append(pts, np.ones((rows, 1)), axis=1)
    return pts

In [280]:
def Fiducial_matrix(X_pts,x_pts):
    rows=X_pts.shape[0]
    X_pts_homo=homo_coord(X_pts)
    x_pts_homo=homo_coord(x_pts)

    T=normalizing_matrix(x_pts)
    U=normalizing_matrix(X_pts)
    # print(T)
    # print(U)
    x_pts_homo=(np.dot(T,x_pts_homo.T)).T
    X_pts_homo=(np.dot(U,X_pts_homo.T)).T

    
    zerocol=np.zeros((rows,4))
    multx=(-X_pts_homo.T*x_pts_homo[:,0]).T
    multy=(-X_pts_homo.T*x_pts_homo[:,1]).T

    P=np.append(X_pts_homo,zerocol,axis=1)
    P=np.append(P,multx,axis=1)
    P=np.append(P,zerocol,axis=1)
    P=np.append(P,X_pts_homo,axis=1)
    P=np.append(P,multy,axis=1)
    
    P=np.reshape(P,(2*rows,12))
    return P

In [281]:
Fiducial_matrix(X,x)

array([[-1.53156745, -1.11386724, -0.97463383,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        , -2.55223374, -1.85616999,
        -1.62414874,  1.66641942],
       [ 0.        ,  0.        ,  0.        ,  0.        , -1.53156745,
        -1.11386724, -0.97463383,  1.        ,  1.11173997,  0.80853816,
         0.70747089, -0.72588378],
       [-0.69616702, -0.27846681, -0.97463383,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        , -0.51674349, -0.2066974 ,
        -0.72344089,  0.74226942],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0.69616702,
        -0.27846681, -0.97463383,  1.        ,  0.36160637,  0.14464255,
         0.50624892, -0.51942473],
       [-0.69616702,  1.39233405, -0.97463383,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.3798578 , -0.75971559,
         0.53180092, -0.54564176],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0.69616702,
         1.39233405, -

In [282]:
def denormalize(X_pts,x_pts,P):
    T=normalizing_matrix(x_pts)
    U=normalizing_matrix(X_pts)
    P=np.dot(inv(T),P)
    P=np.dot(P,U)
    return P

In [283]:
def P_estimate(X_pts,x_pts):
    P=Fiducial_matrix(X_pts,x_pts)
    u, s, vh = np.linalg.svd(P)
    least_val_vector=vh[-1]
    P=np.reshape(least_val_vector,(3,4))
    P=denormalize(X_pts,x_pts,P)
    return P

In [284]:
def P_decomposition(P):
    KR=P[:,:3]
    KRX=P[:,-1]
    r, q = rq(KR)
    X_0 = np.dot(-np.linalg.inv(KR),KRX)
    return r,q,X_0
    # print(r)
    # print(q)

In [285]:
P=P_estimate(X,x)
print(P)

[[-8.84423195e+01  2.71582368e+01  8.40089891e+01  2.76823360e+02]
 [-1.48643609e+02  4.82865902e+01  1.48597337e+02  4.36515931e+02]
 [-1.75825963e-01  5.80696617e-02  1.72788662e-01  5.06063136e-01]]


In [286]:
P_decomposition(P)

(array([[  -2.41916669,   -0.88464024, -124.94194252],
        [   0.        ,    2.15613232, -215.64562801],
        [   0.        ,    0.        ,   -0.25326424]]),
 array([[ 0.52304856,  0.81187109,  0.25939455],
        [ 0.49442102, -0.53692995,  0.68355986],
        [ 0.69423919, -0.22928488, -0.68224657]]),
 array([ 6.47112732, 10.83347145,  0.01523845]))

In [287]:
X_test = np.array([
    [0, 0, 0],
    [11.85, 9.48, 0],
    [4.74, 14.22, 0],
    [11.85, 14.22, 7.11],
    [9.48, 9.48, 0],
    [11.85, 9.48, 2.37]
]
)

In [288]:
x_actual = np.array([
    [124, 887],
    [542, 750],
    [632, 980],
    [789, 694],
    [530, 798],
    [580, 705]
])

In [289]:
def test_pred(P, X_test):
  rows=X_test.shape[0]
  X_test = np.append(X_test, np.ones((rows,1)), axis=1)
  # print(X_test)
  x_test = np.dot(P,X_test.T)
  x_test = x_test/x_test[2]
  x_test = x_test[0:2].T
  return x_test

In [290]:
x_predict=test_pred(P,X_test)

In [291]:
def rms_error(x_predict, x_actual):
    return np.sqrt(np.mean((x_predict-x_actual)**2))

In [292]:
rms_error(x_predict,x_actual)

184.09147086902425